# Earth Engine Tile Layers



## Authentication

The first step is to choose a project and login to Google Cloud.

First install ee api if not already installed
`pip install eartheninge-api`


## Define service account credentials

The service account key should be saved in the `.auth` folder. For security, it is not saved on the repo. 

In [21]:
import json
import ee
service_account = 'tnc-data-v1@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '../.auth/tncKey.json')
ee.Initialize(credentials)

## Test the API by printing the elevation of Mount Rainer

In [22]:
mt_rainer = ee.Geometry.Point([-121.756163642,46.85166326])
img = ee.Image("USGS/NED")
elevation_meters = (img.reduceRegion(ee.Reducer.first(),mt_rainer))
print(elevation_meters.getInfo())

{'elevation': 4312.91259765625}


set up url for tiles 

In [23]:
project_folder = "projects/ee-tacoma-watershed/assets/Tacoma_watershed_assets/"



In [24]:
##get info from raster dictionary
f = open("raster_dict.json",) 
layers = json.load(f)

In [25]:
layer_names = layers.keys()
layer_names


dict_keys(['CanopyClasses', 'lidar_dtm', 'lidar_dsm', 'tacoma_lc_1m_2', 'urban_heat_tac_af', 'urban_heat_tac_pm'])

**Info on visualization parameters**

```params	ImageVisualizationParameters	
The visualization parameters as a (client-side) JavaScript object. For Images and ImageCollections:

  - image (JSON string) The image to render.

  - version (number) Version number of image (or latest).

  - bands (comma-separated strings) Comma-delimited list of band names to be mapped to RGB.

  - min (comma-separated numbers) Value (or one per band) to map onto 00.

  - max (comma-separated numbers) Value (or one per band) to map onto FF.

  - gain (comma-separated numbers) Gain (or one per band) to map onto 00-FF.

  - bias (comma-separated numbers) Offset (or one per band) to map onto 00-FF.

  - gamma (comma-separated numbers) Gamma correction factor (or one per band)

  - palette (comma-separated strings) List of CSS-style color strings (single-band previews only).

  - opacity (number) a number between 0 and 1 for opacity.

  - format (string) Either "jpg" or "png".```

# Test out on a plotly map

In [26]:
import pandas as pd
Tacoma_center = pd.DataFrame({"lat":47.25632945593931,"lon":-122.435107421875},index=[0])
import plotly 
import plotly.express as px

fig = px.scatter_mapbox(Tacoma_center, lat="lat", lon="lon", zoom=8, height=300)

              
# fig.update_layout(
#     mapbox_style="open-street-map",
#     mapbox_layers=[
#         {
#             "sourcetype": "raster",
#             "sourceattribution": "City of Tacoma",
#             "source": [
#                 tiles
#             ]
#         }
#       ])

# fig.show()

NameError: name 'tiles' is not defined

## List all Tacoma Assets 

To get a list of all the assets run `ee.data.listAssets()`

In [28]:
ee.data.listAssets({'parent': 'projects/ee-tacoma-watershed/assets/raster/'})

{'assets': [{'type': 'IMAGE',
   'name': 'projects/ee-tacoma-watershed/assets/raster/canopy_classes',
   'id': 'projects/ee-tacoma-watershed/assets/raster/canopy_classes'},
  {'type': 'IMAGE',
   'name': 'projects/ee-tacoma-watershed/assets/raster/lidar_dsm',
   'id': 'projects/ee-tacoma-watershed/assets/raster/lidar_dsm'},
  {'type': 'IMAGE',
   'name': 'projects/ee-tacoma-watershed/assets/raster/lidar_dtm',
   'id': 'projects/ee-tacoma-watershed/assets/raster/lidar_dtm'},
  {'type': 'IMAGE',
   'name': 'projects/ee-tacoma-watershed/assets/raster/tacoma_lc_1m_2',
   'id': 'projects/ee-tacoma-watershed/assets/raster/tacoma_lc_1m_2'},
  {'type': 'IMAGE',
   'name': 'projects/ee-tacoma-watershed/assets/raster/tnc_runoff_mm',
   'id': 'projects/ee-tacoma-watershed/assets/raster/tnc_runoff_mm'},
  {'type': 'IMAGE',
   'name': 'projects/ee-tacoma-watershed/assets/raster/tnc_tss_ug_L',
   'id': 'projects/ee-tacoma-watershed/assets/raster/tnc_tss_ug_L'},
  {'type': 'IMAGE',
   'name': 'projec

## Render a tile map with vectors

In [29]:
subwatersheds = ee.FeatureCollection("projects/ee-tacoma-watershed/assets/vector/Tacoma_SWPolys_swSubbasin")

map_id_dict = (subwatersheds.getMapId())
vectors=map_id_dict['tile_fetcher'].url_format

fig.update_layout(
    
    mapbox_layers=[
        {
            "sourcetype": "raster",
            "sourceattribution": "City of Tacoma",
            "source": [
                vectors
            ]
        }
      ])

fig.show()


## Request the Feature Collection Data

In [30]:
subwatersheds.getInfo()

{'type': 'FeatureCollection',
 'columns': {'BASINNAME': 'String',
  'LASTDB_EDI': 'String',
  'LASTDB_UPD': 'String',
  'OBJECTID': 'Long',
  'SHAPE_Area': 'Float',
  'SHAPE_Leng': 'Float',
  'SUBBASIN': 'String',
  'created_da': 'String',
  'created_us': 'String',
  'last_edi_1': 'String',
  'last_edite': 'String',
  'system:index': 'String'},
 'version': 1634936908962815,
 'id': 'projects/ee-tacoma-watershed/assets/vector/Tacoma_SWPolys_swSubbasin',
 'properties': {'system:asset_size': 126421},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.3982433645429, 47.24461315334856],
      [-122.3969118358345, 47.2446081965608],
      [-122.39690521177484, 47.24460829855697],
      [-122.39689858771538, 47.24460840055263],
      [-122.39648539677836, 47.244606212414695],
      [-122.39648554670894, 47.24461073269117],
      [-122.39647364974488, 47.245017368266616],
      [-122.39646712808315, 47.24501743782726],
      [-122.39433540373939, 47.

## Get TSS concentration

In [31]:
TSS = ee.Image("projects/ee-tacoma-watershed/assets/raster/tnc_tss_ug_L")
Q = ee.Image("projects/ee-tacoma-watershed/assets/raster/tnc_runoff_mm")
basin = subwatersheds.first()


In [32]:

basin_geojson =basin.getInfo()

basin_geojson

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-122.3982433645429, 47.24461315334856],
    [-122.3969118358345, 47.2446081965608],
    [-122.39690521177484, 47.24460829855697],
    [-122.39689858771538, 47.24460840055263],
    [-122.39648539677836, 47.244606212414695],
    [-122.39648554670894, 47.24461073269117],
    [-122.39647364974488, 47.245017368266616],
    [-122.39646712808315, 47.24501743782726],
    [-122.39433540373939, 47.24501215129513],
    [-122.39433515159443, 47.245007663375645],
    [-122.39434706932575, 47.244601027818575],
    [-122.39251053715095, 47.24459536264897],
    [-122.39153979534358, 47.24459333337441],
    [-122.39104783994756, 47.24459244585799],
    [-122.38980824983885, 47.244590377774145],
    [-122.38978855609844, 47.24459070896753],
    [-122.38890302063878, 47.244587288964986],
    [-122.3859374023869, 47.24455518315145],
    [-122.38587842466664, 47.24455614171296],
    [-122.38539947932635, 47.24455053218936],
    [-122.

In [33]:
tss_concentration = TSS.reduceRegion(
reducer=ee.Reducer.mean(),
geometry =  basin.geometry(), 
scale =  10
)

print(tss_concentration.getInfo())

{'TSS_ug_L': 36806.976085837996}


## get Runoff in mm/yr


In [34]:
avg_runoff = Q.reduceRegion(
reducer=ee.Reducer.mean(),
geometry =  basin.geometry(), 
scale =  10
)

print(avg_runoff.getInfo())
print(avg_runoff.getInfo())

{'mean_annual_runoff_mm': 573.1793828762566}
{'mean_annual_runoff_mm': 573.1793828762566}


In [36]:
#multiply by area to get volume 

#annual_volume = avg_runoff.Multiply(basin.area())
area = basin.geometry().area().getInfo()
annual_mm_val = avg_runoff.getInfo().get('mean_annual_runoff_mm')
runoff_volume = (annual_mm_val)*((area)) #mm/yr * m2 = L/yr
print(runoff_volume, 'liters')



990528588.7217005 liters
